# Keras Callbacks

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import io
from PIL import Image

In [2]:
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping, LearningRateScheduler, ModelCheckpoint, CSVLogger, ReduceLROnPlateau
%load_ext tensorboard

In [3]:
import os
import numpy as np
import math
import datetime
import pandas as pd

In [4]:
print("Version: ", tf.__version__)
tf.get_logger().setLevel('INFO')

Version:  2.2.0


## Keras Callback Applications

In [5]:
splits, info = tfds.load(
    'horses_or_humans', 
    as_supervised=True, 
    with_info=True, 
    split=['train[:80%]', 'train[80%:]', 'test']
)
(train_examples, validation_examples, test_examples) = splits

num_examples = info.splits['train'].num_examples
num_classes = info.features['label'].num_classes

In [6]:
SIZE = 150
IMAGE_SIZE = (SIZE, SIZE)

In [7]:
def format_image(image, label):
    image = tf.image.resize(image, IMAGE_SIZE) / 255.0
    return image, label

In [8]:
BATCH_SIZE = 32

In [9]:
train_batches = train_examples.shuffle(num_examples // 4).map(format_image).batch(BATCH_SIZE).prefetch(1)
validation_batches = validation_examples.map(format_image).batch(BATCH_SIZE).prefetch(1)
test_batches = test_examples.map(format_image).batch(1)

In [10]:
for image_batch, label_batch in train_batches.take(1):
    pass

image_batch.shape

TensorShape([32, 150, 150, 3])

In [11]:
def build_model(dense_units, input_shape=IMAGE_SIZE + (3,)):
    inp = tf.keras.Input(input_shape)
    
    x = tf.keras.layers.Conv2D(16, (3, 3), activation='relu')(inp)
    x = tf.keras.layers.MaxPooling2D(2, 2)(x)
    x = tf.keras.layers.Conv2D(32, (3, 3), activation='relu')(x)
    x = tf.keras.layers.MaxPooling2D(2, 2)(x)
    x = tf.keras.layers.Conv2D(64, (3, 3), activation='relu')(x)
    x = tf.keras.layers.MaxPooling2D(2, 2)(x)
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(dense_units, activation='relu')(x)
    outp = tf.keras.layers.Dense(2, activation='softmax')(x)
    
    model = tf.keras.Model(inp, outp,)
    
    return model

In [ ]:
!rm -rf logs

In [ ]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

logdir = os.path.join('logs', datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir)

model.fit(
    train_batches,
    epochs=10,
    validation_data=validation_batches,
    callbacks=[tensorboard_callback]
)

In [12]:
%tensorboard --logdir logs

/Users/shankar/dev/tools/anaconda3/envs/od/lib/python3.8/site-packages/google/colab/data_table.py:30: UserWarning: IPython.utils.traitlets has moved to a top-level traitlets package.
  from IPython.utils import traitlets as _traitlets


Reusing TensorBoard on port 6006 (pid 6724), started 0:06:57 ago. (Use '!kill 6724' to kill it.)

<IPython.core.display.Javascript object>

In [25]:
!sudo kill -9 6724

Password:


## Model Checkpoint

In [13]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

model.fit(
    train_batches,
    validation_data=validation_batches,
    verbose=2,
    callbacks=[ModelCheckpoint('weights.{epoch:02d}-{val_loss:.2f}.h5', verbose=1)]
)


Epoch 00001: saving model to weights.01-0.67.h5
26/26 - 6s - loss: 0.6773 - accuracy: 0.5669 - val_loss: 0.6708 - val_accuracy: 0.5122


In [14]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=1, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[ModelCheckpoint('saved_model', verbose=1)
          ])


Epoch 00001: saving model to saved_model
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


INFO:tensorflow:Assets written to: saved_model/assets


INFO:tensorflow:Assets written to: saved_model/assets


26/26 - 8s - loss: 0.6584 - accuracy: 0.5985 - val_loss: 0.6147 - val_accuracy: 0.7415


In [15]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(
    train_batches, 
    epochs=2, 
    validation_data=validation_batches, 
    verbose=2,
    callbacks=[
        ModelCheckpoint('model.h5', verbose=1)
    ]
)

Epoch 1/2

Epoch 00001: saving model to model.h5
26/26 - 7s - loss: 0.6762 - accuracy: 0.5949 - val_loss: 0.6730 - val_accuracy: 0.4780
Epoch 2/2

Epoch 00002: saving model to model.h5
26/26 - 6s - loss: 0.6347 - accuracy: 0.6788 - val_loss: 0.6165 - val_accuracy: 0.7512


## Early Stopping

In [16]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(
    train_batches,
    epochs=50,
    validation_data=validation_batches,
    verbose=2,
    callbacks=[
        EarlyStopping(
            patience=3,
            min_delta=0.05,
            baseline=0.8,
            mode='min',
            monitor='val_loss',
            restore_best_weights=True,
            verbose=1
        )
    ]
)    
    

Epoch 1/50
26/26 - 6s - loss: 0.6776 - accuracy: 0.5827 - val_loss: 0.6761 - val_accuracy: 0.4488
Epoch 2/50
26/26 - 7s - loss: 0.6340 - accuracy: 0.6618 - val_loss: 0.6468 - val_accuracy: 0.5512
Epoch 3/50
26/26 - 7s - loss: 0.6002 - accuracy: 0.7068 - val_loss: 0.6206 - val_accuracy: 0.6000
Epoch 4/50
26/26 - 6s - loss: 0.5414 - accuracy: 0.7530 - val_loss: 0.5018 - val_accuracy: 0.8000
Epoch 5/50
26/26 - 6s - loss: 0.4803 - accuracy: 0.8005 - val_loss: 0.4179 - val_accuracy: 0.8732
Epoch 6/50
26/26 - 7s - loss: 0.4360 - accuracy: 0.8187 - val_loss: 0.4114 - val_accuracy: 0.8439
Epoch 7/50
26/26 - 6s - loss: 0.3672 - accuracy: 0.8577 - val_loss: 0.3583 - val_accuracy: 0.8683
Epoch 8/50
26/26 - 6s - loss: 0.3146 - accuracy: 0.8869 - val_loss: 0.2233 - val_accuracy: 0.9659
Epoch 9/50
26/26 - 6s - loss: 0.2410 - accuracy: 0.9173 - val_loss: 0.1796 - val_accuracy: 0.9463
Epoch 10/50
26/26 - 6s - loss: 0.1827 - accuracy: 0.9538 - val_loss: 0.2598 - val_accuracy: 0.8780
Epoch 11/50
26/26 -

## CSV Logger

In [17]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
csv_file = 'training.csv'

model.fit(
    train_batches, 
    epochs=5, 
    validation_data=validation_batches, 
    callbacks=[
        CSVLogger(csv_file)
    ]
)

Epoch 1/5
26/26 [==============================] - 7s 261ms/step - loss: 0.6666 - accuracy: 0.6071 - val_loss: 0.6506 - val_accuracy: 0.7512
Epoch 2/5
26/26 [==============================] - 7s 267ms/step - loss: 0.6212 - accuracy: 0.7263 - val_loss: 0.5912 - val_accuracy: 0.8341
Epoch 3/5
26/26 [==============================] - 7s 265ms/step - loss: 0.5522 - accuracy: 0.7737 - val_loss: 0.5160 - val_accuracy: 0.7805
Epoch 4/5
26/26 [==============================] - 8s 293ms/step - loss: 0.4836 - accuracy: 0.7774 - val_loss: 0.4721 - val_accuracy: 0.8341
Epoch 5/5
26/26 [==============================] - 8s 303ms/step - loss: 0.4440 - accuracy: 0.8005 - val_loss: 0.5942 - val_accuracy: 0.6683


In [18]:
pd.read_csv(csv_file).head()

,epoch,accuracy,loss,val_accuracy,val_loss
0,0,0.607056,0.666567,0.751220,0.650565
1,1,0.726277,0.621224,0.834146,0.591156
2,2,0.773723,0.552151,0.780488,0.516037
3,3,0.777372,0.483584,0.834146,0.472144
4,4,0.800487,0.443994,0.668293,0.594184


## Learning Rate Scheduler

In [20]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])

def step_decay(epoch):
    initial_lr = 0.01
    drop = 0.5
    epochs_drop = 1
    lr = initial_lr * math.pow(drop, math.floor((1 + epoch) / epochs_drop))
    
    return lr

model.fit(
    train_batches,
    epochs=5,
    validation_data=validation_batches,
    callbacks=[
        LearningRateScheduler(
            step_decay,
            verbose=1
        ),
        TensorBoard(log_dir="./logs_dir")
    ]
)


Epoch 00001: LearningRateScheduler reducing learning rate to 0.005.
Epoch 1/5
26/26 [==============================] - 7s 260ms/step - loss: 0.6609 - accuracy: 0.6083 - val_loss: 0.6654 - val_accuracy: 0.5171 - lr: 0.0050

Epoch 00002: LearningRateScheduler reducing learning rate to 0.0025.
Epoch 2/5
26/26 [==============================] - 7s 260ms/step - loss: 0.6238 - accuracy: 0.6837 - val_loss: 0.6162 - val_accuracy: 0.7707 - lr: 0.0025

Epoch 00003: LearningRateScheduler reducing learning rate to 0.00125.
Epoch 3/5
26/26 [==============================] - 7s 251ms/step - loss: 0.6056 - accuracy: 0.7494 - val_loss: 0.6099 - val_accuracy: 0.7366 - lr: 0.0012

Epoch 00004: LearningRateScheduler reducing learning rate to 0.000625.
Epoch 4/5
26/26 [==============================] - 7s 262ms/step - loss: 0.5956 - accuracy: 0.7591 - val_loss: 0.6137 - val_accuracy: 0.7122 - lr: 6.2500e-04

Epoch 00005: LearningRateScheduler reducing learning rate to 0.0003125.
Epoch 5/5
26/26 [========

In [22]:
%tensorboard --logdir log_dir

<IPython.core.display.Javascript object>

## Reduce Learning Rate on Plateau

In [23]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])

model.fit(
    train_batches,
    epochs=50,
    validation_data=validation_batches,
    callbacks=[
        ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.2,
            verbose=1,
            patience=1,
            min_lr=0.001
        ),
        TensorBoard(log_dir='./log_dir')
    ]
)

Epoch 1/50
26/26 [==============================] - 7s 267ms/step - loss: 0.6595 - accuracy: 0.5876 - val_loss: 0.6486 - val_accuracy: 0.5610 - lr: 0.0100
Epoch 2/50
26/26 [==============================] - 7s 261ms/step - loss: 0.5983 - accuracy: 0.6800 - val_loss: 0.5825 - val_accuracy: 0.7415 - lr: 0.0100
Epoch 3/50
26/26 [==============================] - 6s 248ms/step - loss: 0.5270 - accuracy: 0.7567 - val_loss: 0.5127 - val_accuracy: 0.7902 - lr: 0.0100
Epoch 4/50
26/26 [==============================] - 7s 255ms/step - loss: 0.5066 - accuracy: 0.7579 - val_loss: 0.4628 - val_accuracy: 0.7951 - lr: 0.0100
Epoch 5/50
26/26 [==============================] - 6s 245ms/step - loss: 0.4357 - accuracy: 0.7895 - val_loss: 0.4027 - val_accuracy: 0.8293 - lr: 0.0100
Epoch 6/50
26/26 [==============================] - ETA: 0s - loss: 0.3869 - accuracy: 0.8212
Epoch 00006: ReduceLROnPlateau reducing learning rate to 0.0019999999552965165.
26/26 [==============================] - 7s 253ms/s

In [26]:
%tensorboard --logdir log_dir

<IPython.core.display.Javascript object>